In [2]:
import sys 
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torchvision 
import torchvision.transforms as transforms 
import torch.nn.functional as F 
import numpy as np 
import pandas as pd 

from skorch import NeuralNetClassifier 
from time import time 
from tqdm import tqdm 
from torch.utils.data import DataLoader

In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5), (0.5))])

train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

batch_size = 64 

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

100%|██████████| 26.4M/26.4M [00:09<00:00, 2.76MB/s]


Extracting ./data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./data\FashionMNIST\raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 160kB/s]


Extracting ./data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw



100%|██████████| 4.42M/4.42M [00:03<00:00, 1.17MB/s]


Extracting ./data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./data\FashionMNIST\raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 5.58MB/s]


Extracting ./data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw



In [4]:
np.random.seed(20)
torch.manual_seed(20)

In [11]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x1 = torch.relu(self.fc1(x))
        x2 = torch.relu(self.fc2(x1))
        x3 = torch.relu(self.fc3(x2))
        x4 = self.fc4(x3)

        return x1,x2, x3, x4

In [14]:
def evaluate(model):
    model.eval() 
    correct = 0 
    total = 0 
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data 
            outputs = model(inputs)[-1]
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total 

In [15]:
net = SimpleNet()
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(net.parameters(), lr=0.001)

for epoch in range(20):
    net.train() 
    running_loss = 0.0 

    for data in trainloader:
        inputs, labels = data 
        optimizer.zero_grad() 
        outputs = net(inputs)
        loss = criterion(outputs[-1], labels)
        loss.backward()
        optimizer.step() 

        running_loss += loss.item()

    accuracy = evaluate(net) 

    print(f"Epoch {epoch + 1}, Loss: {round(running_loss / len(trainloader), 2)}, Accuracy: {accuracy * 100:.2f}%")

Epoch 1, Loss: 0.5, Accuracy: 84.34%
Epoch 2, Loss: 0.37, Accuracy: 85.70%
Epoch 3, Loss: 0.33, Accuracy: 86.47%
Epoch 4, Loss: 0.31, Accuracy: 87.80%
Epoch 5, Loss: 0.29, Accuracy: 87.84%
Epoch 6, Loss: 0.27, Accuracy: 87.40%
Epoch 7, Loss: 0.26, Accuracy: 88.07%
Epoch 8, Loss: 0.24, Accuracy: 88.06%
Epoch 9, Loss: 0.23, Accuracy: 88.23%
Epoch 10, Loss: 0.22, Accuracy: 87.73%
Epoch 11, Loss: 0.21, Accuracy: 88.35%
Epoch 12, Loss: 0.2, Accuracy: 88.80%
Epoch 13, Loss: 0.19, Accuracy: 88.47%
Epoch 14, Loss: 0.18, Accuracy: 88.64%
Epoch 15, Loss: 0.17, Accuracy: 89.21%
Epoch 16, Loss: 0.17, Accuracy: 88.55%
Epoch 17, Loss: 0.16, Accuracy: 89.28%
Epoch 18, Loss: 0.15, Accuracy: 88.63%
Epoch 19, Loss: 0.14, Accuracy: 88.82%
Epoch 20, Loss: 0.14, Accuracy: 89.61%


In [17]:
net.eval() 

with torch.no_grad():
    for data in testloader:
        inputs, labels = data 
F.softmax(net(inputs[0])[-1])

C:\Users\ungdu\AppData\Local\Temp\ipykernel_8132\1345615224.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(net(inputs[0])[-1])


tensor([[3.0038e-03, 2.2447e-06, 1.0279e-03, 9.8587e-01, 2.0068e-06, 8.4903e-10,
         1.0092e-02, 1.9513e-11, 6.8100e-07, 9.6484e-10]],
       grad_fn=<SoftmaxBackward0>)

In [19]:
net = SimpleNet() 
criterion = nn.CrossEntropyLoss(label_smoothing = 0.2)
optimizer = optim.Adam(net.parameters(), lr=0.001)

for epoch in range(20):
    net.train() 
    running_loss = 0.0 

    for data in trainloader:
        inputs, labels = data 
        optimizer.zero_grad() 
        outputs = net(inputs)
        loss = criterion(outputs[-1], labels)
        loss.backward()
        optimizer.step() 

        running_loss += loss.item()
    accuracy = evaluate(net)

    print(f"Epoch {epoch + 1}, Loss: {round(running_loss / len(trainloader), 2)}, Accruacy: {accuracy * 100:.2f}%")

Epoch 1, Loss: 1.19, Accruacy: 85.12%
Epoch 2, Loss: 1.1, Accruacy: 86.00%
Epoch 3, Loss: 1.08, Accruacy: 87.48%
Epoch 4, Loss: 1.06, Accruacy: 86.97%
Epoch 5, Loss: 1.05, Accruacy: 87.97%
Epoch 6, Loss: 1.04, Accruacy: 86.90%
Epoch 7, Loss: 1.03, Accruacy: 87.68%
Epoch 8, Loss: 1.02, Accruacy: 88.91%
Epoch 9, Loss: 1.01, Accruacy: 88.72%
Epoch 10, Loss: 1.01, Accruacy: 88.20%
Epoch 11, Loss: 1.0, Accruacy: 88.54%
Epoch 12, Loss: 0.99, Accruacy: 88.75%
Epoch 13, Loss: 0.99, Accruacy: 88.76%
Epoch 14, Loss: 0.98, Accruacy: 88.30%
Epoch 15, Loss: 0.98, Accruacy: 88.97%
Epoch 16, Loss: 0.97, Accruacy: 89.01%
Epoch 17, Loss: 0.97, Accruacy: 89.27%
Epoch 18, Loss: 0.96, Accruacy: 88.86%
Epoch 19, Loss: 0.96, Accruacy: 89.19%
Epoch 20, Loss: 0.96, Accruacy: 88.94%


In [20]:
net.eval() 

with torch.no_grad():
    for data in testloader:
        inputs, labels = data 

F.softmax(net(inputs[0])[-1])

C:\Users\ungdu\AppData\Local\Temp\ipykernel_8132\2356410437.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(net(inputs[0])[-1])


tensor([[0.0210, 0.0185, 0.0239, 0.8123, 0.0194, 0.0191, 0.0261, 0.0199, 0.0190,
         0.0209]], grad_fn=<SoftmaxBackward0>)